### User: Data Scientist

#### Goal
- Select and load trade datasets from two different domains
- Filter out the required columns
- Perform a join between the two datasets (This join is backed by SMPC)

#### Summary:
- Select a network
- Login into the network
- Select and ETL the canada trade dataset
- Select and ETL the us trade dataset
- Perform a join between the canada trade dataset and the italy trade dataset on the `Commodity` or `Commodity Code` column

In [ ]:
import syft as sy

# Select the united nations network
un_network = sy.network[0]

# Login into the network
un_network_client = un.login(email="sheldon@caltech.edu", password="bazinga")

In [7]:
# Let's quickly check the datasets available on the network
un_network_client.datasets

,Name,Tags,Description,Dtype,Id,Domain,Shape
0,breast_cancer,"[mri, breast cancer, dicoms]",Labelled image dataset of patients suffering d...,ImageClassificationDataset,56lkw24,WHO,"((25000, 300, 300), (25000))"
1,canada_trade_data,"[canada, trade, un, commodities]",This dataset represents aggregated trade stati...,DataFrame,f3s9h1m,Canada,"(25000, 22)"
2,netherlands_trade_data,"[netherlands, trade, commodities, export]",This dataset represents aggregated trade stati...,DataFrame,2kf3o5d,Netherlands,"(35000, 22)"
3,italy_trade_data,"[italy, trade, un, commodities, export, import]",This dataset represents aggregated trade stati...,DataFrame,42wk65l,Italy,"(30000, 22)"
4,us_trade_data,"[us, trade, un, commodities]",This dataset represents aggregated trade stati...,DataFrame,86pfgh1,United States,"(40000, 22)"


In [ ]:
# Filter and select the Canada and the Italy trade datasets

ca_trade_dataset_ptr = un_network_client.datasets["f3s9h1m"]
it_trade_dataset_ptr = un_network_client.datasets["42wk65l"]

In [ ]:
# As the Data Scientist, we want to return a list of commodities 
# where the ratio of expected imports / exports is off by 10% or more.

# In order to achieve the above, we don't need all the columns of the dataset. 
# Let's filter out the data for the columns we desire.

required_columns = ["Classification", "Commodity Code", "Commodity", "Trade Value (US$)", "Partner", "Commodity Code",  "Trade Flow"]

ca_dataset_ptr = ca_trade_dataset_ptr.select(columns=required_columns)
it_dataset_ptr = it_trade_dataset_ptr.select(columns=required_columns)

# In canada dataset filter out the rows where the `Partner` is `Italy`
ca_filtered_dataset_ptr = ca_dataset_ptr.filter(ca_filtered_dataset_ptr["Partner"] == "Italy")

# Similary, in italy dataset filter out the rows where the `Partner` is `Canada`
it_filtered_dataset_ptr = it_dataset_ptr.filter(ca_filtered_dataset_ptr["Partner"] == "Canada")

In [8]:
# Now, in order to compare the commodities import/export ratios between Canada and Italy,
# we need to create an intermediate table/dataset which joins over the Commodity column.
# Thus, this intermediate table/dataset will contain information about both the import trade value and export trade value 
# for each commodity that was traded between `Canada` and `Italy`.

In [14]:
# Let's join/merge the canada and italy dataset over the `Commodity Code` column.

import sympc

'''
def merge(
    left: [Pointer],
    right: [Pointer],
    how: str = "inner",
    on: Union[str, None] = None,
    left_on: Union[[str, None]] = None,
    right_on: Union[[str, None]] = None,
    suffixes:Tuple = ("_x", "_y"),
    parties:Union[List[Client], None] = None,
):
    """
    The join or merge is done over the columns.
    Parameters
    ----------
        left: Pointer
        right: Pointer
        on: Name of the column on which the join will be performed
        how: {'left', 'right', 'outer', 'inner', 'cross'}, default 'inner', similar to SQL.
        suffixes: these are added as suffix to the column names in the merged dataset,
                  if the column name (except the column on which join is performed)
                  in the two pointer being joined is same,
                  default ('_x', '_y')
        left_on: Name of the column on which the join will be performed in the left pointer
        right_on: Name of the column on which the join will be performed in the right pointer
        parties: It is the list of domains/clients across which the data is shared.
                 If parties is `None`, 
                  - then the same will be inferred from the domain of the two pointers. 
                  - If the domain of the pointer is not defined, then an error is thrown.
    Note
    ----
        The arguments `on`, `left_on` and `right_on` cannot be None at the same time.
        Either `on` or `left_on` and `right_on` needs to be specified to indicate the column
        on which the join needs to be performed.
    """
'''

merged_dataset_ptr = sympc.merge(
    left=ca_filtered_dataset_ptr,
    right=it_filtered_dataset_ptr,
    on="Commodity Code",
    how="inner",
    suffixes=("_ca", "_it"),
)

# Or, if the column names differ in the two datasets being joined, then,

merged_dataset_ptr = sympc.merge(
    left=ca_filtered_dataset_ptr,
    right=it_filtered_dataset_ptr,
    how="inner",
    suffixes=("_ca", "_it"),
    left_on="Commodity Code",
    right_on="Commodity Code",
)

In [26]:
# If the column on which join is performed does not exists in both the Pointers

merged_dataset_ptr = sympc.merge(
    left=ca_filtered_dataset_ptr,
    right=it_filtered_dataset_ptr,
    how="inner",
    on='MyColumn'
    suffixes=("_ca", "_it"),
)


 ColumnDoesNotExistError: Column `MyColumn` does not exists in the pointers being joined.



In [27]:
# If the column on which join is performed does not exists in one of the Pointers, let's say the left pointer.

merged_dataset_ptr = sympc.merge(
    left=ca_filtered_dataset_ptr,
    right=it_filtered_dataset_ptr,
    how="inner",
    on='MyColumn'
    suffixes=("_ca", "_it"),
)


 ColumnDoesNotExistError: Column `MyColumn` does not exists in the `left` Pointer.



In [57]:
# If common column names exists in the pointers being joined, and the suffixes are set to None, then
# an error should be thrown as join cannot be created.
merged_dataset_ptr = sympc.merge(
    left=ca_filtered_dataset_ptr,
    right=it_filtered_dataset_ptr,
    how="inner",
    on='Commodity Code'
    suffixes=(None, None),
)


 DuplicateColumnsError:  Duplicates columns are present in the two pointers being joined. 
                         `suffix` cannot be `None` in such case. Please provide a suffix 
                         if you want to distinguish the duplicate columns in the merged dataset.

                         e.g.
                         sympc.merge(left, right, on="CommonColumnName", suffix=("_x", "_y"))



##### Awesome we successfully perform a join operation between the Canada and Italy datasets. Now, we can easily infer the export to import ratios between Canada and Italy from this new formed merged dataset.

In [69]:
# Let's see the columns of the merged dataset
merged_dataset.column_description

,Column,Description,Private
0,Classification_ca,Commodity Classification (HS= Harmonized System),True
1,Commodity Code,HS Commodity Code,True
2,Commodity_ca,Description,True
3,Trade Value_ca,in US dollars,True
4,Partner_ca,Description,False
5,Trade Flow_ca,Description,False
6,Classification_it,Commodity Classification (HS= Harmonized System),True
7,Commodity_it,Description,True
8,Trade Value_it,in US dollars,True
9,Partner_it,Description,True


#### Joining the dataset and other calc -Delete this later

In [ ]:
import pandas as pd

In [ ]:
import sympc
from sympc.session import Session
from sympc.session import SessionManager

from sympc.tensor import MPCTensor

parties = []


mpc_tensor_1 = MPCTensor(secret=value_secret_1, shape=(2,5), parties=parties)

In [ ]:
parties = [canada, us, sheldon]
value_secret_1 = gryffindor.syft.core.tensor.tensor.Tensor(np.array([[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]], dtype=np.int64))


mpc_tensor_1 = MPCTensor(secret=value_secret_1, shape=(2,5), parties=parties)
print(mpc_tensor_1)

mpc_res = mpc_tensor_1.sum(axis=0)
print(mpc_res.reconstruct())

[ 7  9 11 13 15] 

In [1]:
import pandas as pd

ca_data = pd.read_csv("datasets/ca - feb 2021.csv")
it_data = pd.read_csv("datasets/it - feb 2021.csv")

/home/user/.pyenv/versions/3.8.1/envs/syft/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
required_columns = ["Classification", "Commodity Code", "Commodity", "Trade Value (US$)", "Partner", "Commodity Code", "Trade Flow"]

In [11]:
ca_data[required_columns]

,Classification,Commodity Code,Commodity,Trade Value (US$),Partner,Commodity Code,Trade Flow
0,HS,6117,"Clothing accessories; made up, knitted or croc...",9285,"Other Asia, nes",6117,Imports
1,HS,18,Cocoa and cocoa preparations,116604,Egypt,18,Imports
2,HS,18,Cocoa and cocoa preparations,1495175,United Kingdom,18,Imports
3,HS,18,Cocoa and cocoa preparations,2248,United Rep. of Tanzania,18,Imports
4,HS,18,Cocoa and cocoa preparations,47840,Singapore,18,Imports
...,...,...,...,...,...,...,...
227449,HS,550952,"Yarn; (not sewing thread), of polyester staple...",34272,World,550952,Exports
227450,HS,550999,"Yarn; (not sewing thread), of synthetic staple...",228182,World,550999,Exports
227451,HS,550969,"Yarn; (not sewing thread), of acrylic or modac...",18812,World,550969,Exports
227452,HS,550962,"Yarn; (not sewing thread), of acrylic or modac...",23140,World,550962,Exports


In [85]:
ca_data = ca_data[:25000]
it_data = it_data[:25000]

#ca_data.query("Partner == 'Italy'")
#it_data.query("Partner == 'Canada'")

In [86]:
ca_partner_it_data = ca_data.query("Partner == 'Italy'")
ca_partner_it_data.reset_index(inplace=True)

In [87]:
it_partner_ca_data = us_data.query("Partner == 'Canada'")
it_partner_ca_data.reset_index(inplace=True)

In [91]:
it_partner_ca_data.merge(ca_partner_it_data, on='Commodity Code')

,index_x,Classification_x,Year_x,Period_x,Period Desc._x,Aggregate Level_x,Is Leaf Code_x,Trade Flow Code_x,Trade Flow_x,Reporter Code_x,...,Partner Code_y,Partner_y,Partner ISO_y,Commodity_y,Qty Unit Code_y,Qty Unit_y,Qty_y,Netweight (kg)_y,Trade Value (US$)_y,Flag_y
0,196885,HS,2021,202102,February 2021,2,0,1,Imports,842,...,381,Italy,NaN,Fertilizers,0,NaN,NaN,0.0,19940,0
1,343926,HS,2021,202102,February 2021,2,0,3,Re-exports,842,...,381,Italy,NaN,Fertilizers,0,NaN,NaN,0.0,19940,0
2,196982,HS,2021,202102,February 2021,2,0,1,Imports,842,...,381,Italy,NaN,"Essential oils and resinoids; perfumery, cosme...",0,NaN,NaN,NaN,10372371,0
3,344003,HS,2021,202102,February 2021,2,0,3,Re-exports,842,...,381,Italy,NaN,"Essential oils and resinoids; perfumery, cosme...",0,NaN,NaN,NaN,10372371,0
4,197031,HS,2021,202102,February 2021,2,0,1,Imports,842,...,381,Italy,NaN,"Soap, organic surface-active agents; washing, ...",0,NaN,NaN,NaN,1710836,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,359962,HS,2021,202102,February 2021,4,0,3,Re-exports,842,...,381,Italy,NaN,Ceramic articles; n.e.s. in chapter 69,0,NaN,NaN,NaN,203112,0
345,360007,HS,2021,202102,February 2021,4,0,3,Re-exports,842,...,381,Italy,NaN,Statuettes and other ornamental ceramic articles,0,NaN,NaN,NaN,32560,0
346,360232,HS,2021,202102,February 2021,6,1,2,Exports,842,...,381,Italy,NaN,"Skates; ice and roller, including skating boot...",0,NaN,NaN,NaN,4913,0
347,360349,HS,2021,202102,February 2021,6,1,3,Re-exports,842,...,381,Italy,NaN,"Skates; ice and roller, including skating boot...",0,NaN,NaN,NaN,4913,0


#### Dummy Data

In [67]:
import pandas as pd
from enum import Enum


## Dummy Data Store
dataset_store = [
    {
        "Name": "breast_cancer",
        "Tags": ["mri", "breast cancer", "dicoms"],
        "Description": "Labelled image dataset of patients suffering different types of breast cancer",
        "Dtype": "ImageClassificationDataset",
        "Id": "56lkw24",
        "Domain": "WHO",
        "Shape": "((25000, 300, 300), (25000))",
    },
    {
        "Name": "canada_trade_data",
        "Tags": ["canada", "trade", "un", "commodities"],
        "Description": "This dataset represents aggregated trade statistics as reported by Canada about what it believes was imported/exported to/from its country in Feb 2021.",
        "Dtype": "DataFrame",
        "Id": "f3s9h1m",
        "Domain": "Canada",
        "Shape": "(25000, 22)",
    },
    {
        "Name": "netherlands_trade_data",
        "Tags": ["netherlands", "trade", "commodities", "export"],
        "Description": "This dataset represents aggregated trade statistics as reported by Netherlands about what it believes was imported/exported to/from its country in Feb 2021.",
        "Dtype": "DataFrame",
        "Id": "2kf3o5d",
        "Domain": "Netherlands",
        "Shape": "(35000, 22)",
    },
    {
        "Name": "italy_trade_data",
        "Tags": ["italy", "trade", "un", "commodities", "export", "import"],
        "Description": "This dataset represents aggregated trade statistics as reported by Italy about what it believes was imported/exported to/from its country in Feb 2021.",
        "Dtype": "DataFrame",
        "Id": "42wk65l",
        "Domain": "Italy",
        "Shape": "(30000, 22)",
    },
    {
        "Name": "us_trade_data",
        "Tags": ["us", "trade", "un", "commodities"],
        "Description": "This dataset represents aggregated trade statistics as reported by United States about what it believes was imported/exported to/from its country in Feb 2021.",
        "Dtype": "DataFrame",
        "Id": "86pfgh1",
        "Domain": "United States",
        "Shape": "(40000, 22)",
    },
]

dataset_store = pd.DataFrame(dataset_store)

class bcolors(Enum):
    HEADER = "\033[95m"
    OKBLUE = "\033[94m"
    OKCYAN = "\033[96m"
    OKGREEN = "\033[92m"
    WARNING = "\033[93m"
    FAIL = "\033[91m"
    ENDC = "\033[0m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"

    
column_does_not_exists_in_both_error = f'''
{bcolors.FAIL.value} ColumnDoesNotExistError: {bcolors.ENDC.value}Column `MyColumn` does not exists in the pointers being joined.
'''
column_does_not_exists_in_one_error = f'''
{bcolors.FAIL.value} ColumnDoesNotExistError: {bcolors.ENDC.value}Column `MyColumn` does not exists in the `left` Pointer.
'''

duplicate_columns_present_error = f'''
{bcolors.FAIL.value} DuplicateColumnsError: {bcolors.ENDC.value} Duplicates columns are present in the two pointers being joined. 
                         `suffix` cannot be `None` in such case. Please provide a suffix 
                         if you want to distinguish the duplicate columns in the merged dataset.\n
                         e.g.
                         {bcolors.OKCYAN.value}sympc.merge(left, right, on="CommonColumnName", suffix=("_x", "_y"))
'''


# print(column_does_not_exists_in_both_error)
# print(column_does_not_exists_in_one_error)
# print(duplicate_columns_present_error)



# Dummy dataset schema
dataset_schema = pd.read_csv("datasets/schema.csv")

private_values = [
    True,
    False,
    False,
    False,
    True,
    True,
    True,
    True,
    False,
    False,
    False,
    False,
    False,
    False,
    False,
    True,
    True,
    False,
    False,
    False,
    True,
    False,
]

dataset_schema["Private"] = private_values

d={
 'Column': {0: 'Classification_ca',
  1: 'Commodity Code',
  2: 'Commodity_ca',
  3: 'Trade Value_ca',
  4: 'Partner_ca',
  5: 'Trade Flow_ca',
  6: 'Classification_it',
  7: 'Commodity_it',
  8: 'Trade Value_it',
  9: 'Partner_it',
  10: 'Trade Flow_it'},
 'Description': {0: 'Commodity Classification (HS= Harmonized System)',
  1: 'HS Commodity Code',
  2: 'Description',
  3: 'in US dollars',
  4: 'Description',
  5: 'Description',
  6: 'Commodity Classification (HS= Harmonized System)',
  7: 'Description',
  8: 'in US dollars',
  9: 'Description',
  10: 'Description'},
 'Private': {0: True, 1: True, 2: True, 3: True, 4: False, 5: False, 6: True, 7: True, 8: True, 9: True, 10: False}}

merged_dataset_schema = pd.DataFrame.from_dict(d)